In [1]:



import pandas as pd

# Read with proper configuration for your file format
df = pd.read_csv('C:\\YEAR1\\meteo_CG\\nhxnax101_MisureEn_MazPrecO_2_20251121164856.txt', 
                 sep=';',           # Semicolon separator
                 encoding='utf-8',  # Encoding (adjust if needed)
                 decimal=',')       # If decimal values use comma

print(df.head())
print(df.dtypes)



           Stazione                          Grandezza Data rilevazione  \
0  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   
1  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   
2  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   
3  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   
4  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   

  Ora rilevazione  Valore  
0           00:00     0.0  
1           01:00     0.0  
2           02:00     0.0  
3           03:00     0.0  
4           04:00     0.0  
Stazione             object
Grandezza            object
Data rilevazione     object
Ora rilevazione      object
Valore              float64
dtype: object


In [21]:
for i, column in enumerate(df.columns):
    print(f"{i}: {column}")

0: Stazione
1: Grandezza
2: Data rilevazione
3: Ora rilevazione
4: Valore


In [22]:
# Check the actual column names
print("DataFrame columns:")
print(df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

DataFrame columns:
['Stazione', 'Grandezza', 'Data rilevazione', 'Ora rilevazione', 'Valore']

First few rows:
           Stazione                          Grandezza Data rilevazione  \
0  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   
1  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   
2  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   
3  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   
4  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   

  Ora rilevazione  Valore  
0           00:00     0.0  
1           01:00     0.0  
2           02:00     0.0  
3           03:00     0.0  
4           04:00     0.0  


In [23]:
# Manual datetime conversion
df['DataOra'] = pd.to_datetime(
    df['Data rilevazione'] + ' ' + df['Ora rilevazione'], 
    dayfirst=True,  # Important for DD/MM/YYYY format
    format='%d/%m/%Y %H:%M'
)

# Verify datetime conversion
print("After datetime conversion:")
print(df[['Data rilevazione', 'Ora rilevazione', 'DataOra']].head())
print(f"\nDataOra dtype: {df['DataOra'].dtype}")

# Now extract time components
df['Year'] = df['DataOra'].dt.year
df['Month'] = df['DataOra'].dt.month
df['Season'] = df['DataOra'].dt.month % 12 // 3 + 1
df['YearMonth'] = df['DataOra'].dt.to_period('M')

season_map = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Autumn'}
df['Season_Name'] = df['Season'].map(season_map)

print("\nDataFrame with time components:")
print(df.head())

After datetime conversion:
  Data rilevazione Ora rilevazione             DataOra
0       01/01/2023           00:00 2023-01-01 00:00:00
1       01/01/2023           01:00 2023-01-01 01:00:00
2       01/01/2023           02:00 2023-01-01 02:00:00
3       01/01/2023           03:00 2023-01-01 03:00:00
4       01/01/2023           04:00 2023-01-01 04:00:00

DataOra dtype: datetime64[ns]

DataFrame with time components:
           Stazione                          Grandezza Data rilevazione  \
0  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   
1  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   
2  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   
3  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   
4  Mazara del Vallo  Precipitazioni totali orarie (mm)       01/01/2023   

  Ora rilevazione  Valore             DataOra  Year  Month  Season YearMonth  \
0           00:00     0.0 2023-01-01 00:00:00  2

In [25]:
# Filter for precipitation data only
precip_df = df[df['Grandezza'].str.contains('Precipitazioni', na=False)].copy()

print(f"Precipitation measurements: {len(precip_df)}")
print("\nPrecipitation measurement types:")
print(precip_df['Grandezza'].value_counts())

# Calculate monthly precipitation sum and count of valid measurements
monthly_precip = precip_df.groupby(['Stazione', 'Grandezza', 'Year', 'Month']).agg({
    'Valore': ['sum', 'count']
}).round(2)

# Flatten column names
monthly_precip.columns = ['Total_Precipitation', 'Valid_Measurements']
monthly_precip = monthly_precip.reset_index()

print("\nMonthly Precipitation Summary:")
print(monthly_precip.head(14))

Precipitation measurements: 24120

Precipitation measurement types:
Grandezza
Precipitazioni totali orarie (mm)    24120
Name: count, dtype: int64

Monthly Precipitation Summary:
            Stazione                          Grandezza  Year  Month  \
0   Mazara del Vallo  Precipitazioni totali orarie (mm)  2023      1   
1   Mazara del Vallo  Precipitazioni totali orarie (mm)  2023      2   
2   Mazara del Vallo  Precipitazioni totali orarie (mm)  2023      3   
3   Mazara del Vallo  Precipitazioni totali orarie (mm)  2023      4   
4   Mazara del Vallo  Precipitazioni totali orarie (mm)  2023      5   
5   Mazara del Vallo  Precipitazioni totali orarie (mm)  2023      6   
6   Mazara del Vallo  Precipitazioni totali orarie (mm)  2023      7   
7   Mazara del Vallo  Precipitazioni totali orarie (mm)  2023      8   
8   Mazara del Vallo  Precipitazioni totali orarie (mm)  2023      9   
9   Mazara del Vallo  Precipitazioni totali orarie (mm)  2023     10   
10  Mazara del Vallo  Precipi

In [26]:
# Precipitation totals pivot table
precip_totals = monthly_precip.pivot_table(
    index=['Stazione', 'Grandezza', 'Year'],
    columns='Month',
    values='Total_Precipitation',
    aggfunc='sum'
).round(1)

# Valid measurements count pivot table
measurements_count = monthly_precip.pivot_table(
    index=['Stazione', 'Grandezza', 'Year'],
    columns='Month',
    values='Valid_Measurements',
    aggfunc='sum'
)

print("MONTHLY PRECIPITATION TOTALS (mm):")
print("=" * 50)
print(precip_totals.head(20))

print("\nNUMBER OF VALID HOURLY MEASUREMENTS:")
print("=" * 50)
print(measurements_count.head(20))

MONTHLY PRECIPITATION TOTALS (mm):
Month                                                      1      2     3   \
Stazione         Grandezza                         Year                      
Mazara del Vallo Precipitazioni totali orarie (mm) 2023  82.6   40.4  47.8   
                                                   2024  60.6   46.2  42.4   
                                                   2025  75.6  119.4  36.2   

Month                                                      4     5     6   \
Stazione         Grandezza                         Year                     
Mazara del Vallo Precipitazioni totali orarie (mm) 2023  22.4  80.2  22.8   
                                                   2024  13.2  23.4   5.8   
                                                   2025  28.2  42.6  18.4   

Month                                                     7     8      9   \
Stazione         Grandezza                         Year                     
Mazara del Vallo Precipitazioni to